# Qiskit Integration via RFSoCBackend

This notebook demonstrates how to run **Qiskit** quantum circuits on pynq-quantum using the `RFSoCBackend` bridge. This lets you use Qiskit's familiar circuit-building API while executing on RFSoC hardware (or simulation).

**Requires:** `pip install qiskit` (the bridge uses Qiskit's `QuantumCircuit` format)

In [ ]:
from pynq_quantum.bridges.qiskit import RFSoCBackend

# Create an RFSoCBackend using simulation (no hardware needed).
# When no overlay is provided, it creates one automatically.
backend = RFSoCBackend(num_qubits=4)
print(f"Backend name: {backend.name}")
print(f"Max qubits: {backend.num_qubits}")

In [ ]:
try:
    from qiskit import QuantumCircuit as QiskitCircuit

    # Build a Qiskit circuit: 2-qubit Bell state
    qc = QiskitCircuit(2, 2)
    qc.h(0)
    qc.cx(0, 1)
    qc.measure([0, 1], [0, 1])
    print(qc.draw(output="text"))

    # Execute on the RFSoC simulation backend
    job = backend.run(qc, shots=1000)
    print(f"\nJob ID: {job.job_id()}")
    print(f"Status: {job.status()}")

    counts = job.result().get_counts()
    print(f"Counts: {counts}")
    # Expected: roughly 50% '00' and 50% '11'

except ImportError:
    print("Qiskit is not installed. Install with: pip install qiskit")
    print()
    print("Without Qiskit, you can still use pynq-quantum's native")
    print("QuantumCircuit class (see notebooks 01-03).")

## How the Bridge Works

The `RFSoCBackend` implements enough of the Qiskit `BackendV2` interface to be used with `backend.run(circuit)`:

1. **Circuit conversion** -- Qiskit gate names (`h`, `cx`, `rx`, etc.) are mapped to pynq-quantum `GateOp` objects.
2. **Execution** -- The converted operations are run through the simulation backend (state-vector) or compiled to pulse instructions for hardware.
3. **Results** -- An `RFSoCJob` is returned with `.result().get_counts()` matching Qiskit's result format.

Supported Qiskit gates: `x`, `y`, `z`, `h`, `s`, `t`, `cx`, `cz`, `swap`, `rx`, `ry`, `rz`.

To run on real RFSoC hardware, pass an overlay with a hardware backend:
```python
from pynq_quantum import QuantumOverlay
overlay = QuantumOverlay(backend="qick", bitstream="path/to/bitstream.bit")
backend = RFSoCBackend(overlay=overlay)
```